In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


from datetime import date
from sklearn.neighbors import LocalOutlierFactor  #Detección de valores atípicos
from sklearn.preprocessing import LabelEncoder

import os
for dirname, _, filenames in os.walk('data_base'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data_base\US_Accidents_March23.csv


In [4]:
# Cargar el conjunto de datos
data=pd.read_csv('data_base/US_Accidents_March23.csv')

In [5]:
print(data.columns)

Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng', 'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Street', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')


In [6]:
# Retorna las primeras 5 filas del DataFrame
data[['Weather_Condition', 'City', 'Start_Time', 'Severity']].head()
#data.head()

,Weather_Condition,City,Start_Time,Severity
0,Light Rain,Dayton,2016-02-08 05:46:00,3
1,Light Rain,Reynoldsburg,2016-02-08 06:07:59,2
2,Overcast,Williamsburg,2016-02-08 06:49:27,2
3,Mostly Cloudy,Dayton,2016-02-08 07:23:34,3
4,Mostly Cloudy,Dayton,2016-02-08 07:39:07,2


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [8]:
# Convertir la columna 'Start_Time' a tipo datetime y extraer la hora con el formato día/noche
data['Start_Time'] = pd.to_datetime(data['Start_Time'], errors='coerce')
data['Start_Hour'] =  data['Start_Time'].dt.hour
data['Time_Of_Day'] = data['Start_Hour'].apply(lambda x: 'noche' if x < 6 or x >= 18 else 'día')

# Definir las columnas a utilizar
columns_to_use = ['Weather_Condition', 'Time_Of_Day', 'City', 'Severity']
df = data[columns_to_use].copy()  # Extraer esas columnas reales del DataFrame
df.columns = ['Clima', 'Hora_Del_Dia', 'Ciudad', 'Gravedad']  # Renombrar las columnas

# Transformar las columnas categóricas en números 
le_Clima = LabelEncoder()
le_Ciudad = LabelEncoder()
le_Hora_Del_Dia = LabelEncoder()

df['Clima'] = le_Clima.fit_transform(df['Clima'])
df['Ciudad'] = le_Ciudad.fit_transform(df['Ciudad'])
df['Hora_Del_Dia'] = le_Hora_Del_Dia.fit_transform(df['Hora_Del_Dia'])

In [39]:
'''''
print("Codificación de clima:")
for i, clase in enumerate(le_Clima.classes_):
    print(f"{clase} → {i}")

print("\nCodificación de ciudad:")
for i, clase in enumerate(le_Ciudad.classes_):
    print(f"{clase} → {i}")
'''''

'\'\'\nprint("Codificación de clima:")\nfor i, clase in enumerate(le_Clima.classes_):\n    print(f"{clase} → {i}")\n\nprint("\nCodificación de ciudad:")\nfor i, clase in enumerate(le_Ciudad.classes_):\n    print(f"{clase} → {i}")\n'

In [9]:
df.head()

,Clima,Hora_Del_Dia,Ciudad,Gravedad
0,62,1,2894,3
1,62,0,10195,2
2,88,0,13347,2
3,85,0,2894,3
4,85,0,2894,2


In [10]:
# Preparar la matriz para mínimos cuadrados de forma matricial Ax=y
A = df[['Clima', 'Hora_Del_Dia', 'Ciudad']].to_numpy()
n = A.shape[0]
A = np.hstack([A, np.ones((n, 1))])  # Agregar columna de 1s para el término independiente y formar la matriz A

y = df['Gravedad'].to_numpy() # Vector de resultados

In [11]:
def gauss_seidel(A, b, x0=None, tol=1e-8, N=1000):
    """
    Parámetros:
        A : matriz de coeficientes
        b : vector de términos independientes
        x0 : vector inicial (opcional)
        tol : tolerancia para el criterio de parada
        N : número máximo de iteraciones
    Retorna:
        x : solución aproximada
        k : número de iteraciones realizadas
    """
    n = len(b)
    x = x0.copy()
    for k in range(1, N + 1):
        x_new = np.copy(x)
        for i in range(n):
            s1 = sum(A[i, j] * x_new[j] for j in range(i))
            s2 = sum(A[i, j] * x[j] for j in range(i + 1, n))
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            print(f"Convergencia alcanzada en {k} iteraciones.")
            return x_new, k
        x = x_new
    print("Número máximo de iteraciones excedido. El procedimiento no fue exitoso.")
    return x

In [13]:
def matriz_transpuesta(A):
    """
    Parámetro:
        A: matriz de coeficientes
    Retorna:
        transpuesta: matriz transpuesta de A
    """
    if not A or not A[0]:
        return []
    filas = len(A)
    columnas = len(A[0])
    transpuesta = []
    for j in range(columnas):
        nueva_fila = []
        for i in range(filas):
            nueva_fila.append(A[i][j])
        transpuesta.append(nueva_fila)
    return transpuesta

In [16]:
def multiplicar_matrices(A, B):
    """
    Parámetros:
        A: matriz de tamaño m x n
        B: matriz de tamaño n x p
    Retorna:
        resultado: matriz de tamaño m x p
    """
    if not A or not B or len(A[0]) != len(B):
        raise ValueError("Las dimensiones de las matrices no son compatibles para la multiplicación.")
    m, n, p = len(A), len(B), len(B[0])
    resultado = []
    for i in range(m):
        fila = []
        for j in range(p):
            suma = 0
            for k in range(n):
                suma += A[i][k] * B[k][j]
            fila.append(suma)
        resultado.append(fila)
    return resultado

In [21]:
list_A = A.tolist()
list_y = [[val] for val in y]  # convertir y en columna
AT = matriz_transpuesta(list_A) # A^T
ATA = multiplicar_matrices(AT, list_A)  # A^T * A
ATy = multiplicar_matrices(AT, list_y)  # A^T * y

In [ ]:
# Resolver el sistema de ecuaciones normales A^T * A * x = A^T * y
# Tranformación a numpy array
ATA = np.array(ATA, dtype=float)
ATy = np.array(ATy, dtype=float)
x0 = np.zeros(len(ATy))  # Vector inicial de ceros
x, k = gauss_seidel(ATA, ATy, x0=x0)

Convergencia alcanzada en 68 iteraciones.


In [ ]:
clima_nuevo = 3
hora_nueva = 0
ciudad_nueva = 25

# Predicción
gravedad_predicha = (
    x[0] * clima_nuevo +
    x[1] * hora_nueva +
    x[2] * ciudad_nueva +
    x[3]  # término independiente
)

print(f"La gravedad predicha del accidente es aproximadamente: {gravedad_predicha:.2f}")

La gravedad predicha del accidente es aproximadamente: 2.19
